In [ ]:
import contextlib
from io import StringIO
from pathlib import Path

import pandas as pd

import joblib
import category_encoders
from MulticoreTSNE import MulticoreTSNE as TSNE

import plotly.express as px

from jupyter_dash import JupyterDash

import dash_table as dt
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, ALL

import pyskim
from tqdm.auto import tqdm

# Parameters

In [ ]:
fname_in = snakemake.input.fname

model_list = snakemake.params.model_list
dataset_list = snakemake.params.dataset_list

notebook_wd = Path('.')

# Read data

In [ ]:
df = pd.read_csv(fname_in)
df.head()

# Create Dashboard

## Helper functions

In [ ]:
def load_dataset(dataset):
    return pd.read_csv(notebook_wd / 'resources' / 'datasets' / f'{dataset}.csv')

In [ ]:
def do_tsne(df):
    df_sub = df.filter(regex='^(?!target__)')  # remove response
    target = df.filter(regex='^target__').squeeze()
    
    df_sub = category_encoders.OrdinalEncoder().fit_transform(df_sub.fillna(-1))
    
    tsne = TSNE(n_components=2, random_state=0)
    proj = tsne.fit_transform(df_sub)
    
    return tsne, pd.DataFrame(proj, columns=['TSNE_0', 'TSNE_1'], index=target).reset_index()

In [ ]:
def load_model(model, dataset):
    return (
        joblib.load(notebook_wd / 'results' / 'models' / f'{model}__{dataset}.pkl'),
        joblib.load(notebook_wd / 'results' / 'models' / f'encoder_{model}__{dataset}.pkl')
    )

## Setup environment

### Precompute t-SNE projections

In [ ]:
TSNE_CACHE = {}

for dataset in tqdm(dataset_list):
    df_data = load_dataset(dataset)
    model, df_proj = do_tsne(df_data)
    
    TSNE_CACHE[dataset] = {
        'model': model,
        'df_proj': df_proj,
        'df_data': df_data
    }

### Preload models

In [ ]:
MODEL_CACHE = {}

for dataset in tqdm(dataset_list):
    MODEL_CACHE[dataset] = {}  # yeah, defaultdicts exists
    for model in tqdm(model_list, leave=False):
        MODEL_CACHE[dataset][model] = load_model(model, dataset)

## The actual App

In [ ]:
# plotting
def update_figure():
    return px.bar(
        df, x='model', y='accuracy',
        facet_col='dataset',
        barmode='group', title='Overall accuracy'
    )

# create app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1('Auto-ML Benchmark results'),
    dcc.Graph(figure=update_figure()),
    html.Hr(),
    html.Label(f'Select dataset: '),
    dcc.Dropdown(
        id='dataset-selector',
        options=[{'label': name, 'value': name} for name in dataset_list],
        value='iris'
    ),
#     html.Label(id='dataset-skim'),
    html.Hr(),
    html.Div(id='covariate-container'),
    dbc.Row(
        [
            dbc.Col(dcc.Graph(id='projection-container')),
            dbc.Col(dt.DataTable(
                id='prediction-container',
                columns=[
                    {'name': 'Model', 'id': 'model'},
                    {'name': 'Prediction', 'id': 'prediction'},
                    {'name': 'Performance', 'id': 'performance'}
                ],
                sort_action='native'
            ))
        ]
    ),  
])

# plots with callbacks
# @app.callback(
#     Output('dataset-skim', 'children'),
#     [Input('dataset-selector', 'value')]
# )
# def skim_dataset(dataset):
#     df_data = load_dataset(dataset)
    
#     strio = StringIO()
#     with contextlib.redirect_stdout(strio):
#         pyskim.skim(df_data)
#     txt = strio.getvalue()
    
#     return txt

@app.callback(
    Output('covariate-container', 'children'),
    [Input('dataset-selector', 'value')]
)
def assemble_input_fields(dataset):
    df_data = load_dataset(dataset)
    
    children = []
    for col in df_data.columns:
        if col.startswith('target__'):
            continue
            
        exemplary_value = str(df_data[col].iloc[1])
        
        children.extend([
            html.Label(f'{col}: '),
            dcc.Input(
                id={'type': 'covariate', 'name': col},
                placeholder=exemplary_value,
                debounce=False
            ),
            html.Br()
        ])
        
    return children

@app.callback(
    Output('projection-container', 'figure'),
    [
        Input('dataset-selector', 'value'),
        Input({'type': 'covariate', 'name': ALL}, 'value')
    ]
)
def plot_tsne_projection(dataset, *covariates):
    model = TSNE_CACHE[dataset]['model']
    df_proj = TSNE_CACHE[dataset]['df_proj'].copy()
    df_data = TSNE_CACHE[dataset]['df_data'].copy()
    
    cov_columns = df_data.filter(regex='^(?!target__)').columns
    target_column = df_data.filter(regex='^target__').columns[0]
    
    if len(covariates[0]) > 0 and None not in covariates[0]:
        # when adding a new point, we do have to refit the projection
        df_data = df_data.append({
            target_column: 'MANUAL',
            **dict(zip(cov_columns, covariates[0]))
        }, ignore_index=True).apply(pd.to_numeric, errors='ignore')

        _, df_proj = do_tsne(df_data)

    df_agg = pd.concat([df_proj, df_data[cov_columns]], axis=1)
    return px.scatter(
        df_agg,
        x='TSNE_0', y='TSNE_1',
        color=target_column,
        hover_data=cov_columns,
        title='t-SNE projection'
    )

@app.callback(
    Output('prediction-container', 'data'),
    [
        Input('dataset-selector', 'value'),
        Input({'type': 'covariate', 'name': ALL}, 'value')
    ]
)
def plot_model_predictions(dataset, *covariates):
    if len(covariates[0]) == 0 or None in covariates[0]:
        return []
    
    # assmble current input
    df_data = load_dataset(dataset)
    
    # fit models
    data = []
    for model in model_list:
        # model prediction
        clf, enc = MODEL_CACHE[dataset][model]
        
        # prepare data
        df_data = enc.transform(df_data)
    
        cov_columns = df_data.filter(regex='^(?!target__)').columns
        target_column = df_data.filter(regex='^target__').columns[0]

        df_cur = pd.DataFrame({
            **dict(zip(cov_columns, covariates[0]))
        }, index=[0]).apply(
            pd.to_numeric, errors='ignore'
        ).applymap(lambda x: {'True': True, 'False': False}.get(x, x))  # can you see my desperation?
        
        tmp = df_data.filter(regex='^target__').squeeze().loc[:0]
        df_cur = enc.transform(
            pd.concat([df_cur, tmp], axis=1)
        ).drop(columns=tmp.name)
            
        # do prediction
        pred = clf.predict(df_cur)
        
        pred = pd.Series(pred, index=df_cur.index, name=target_column)
        pred = enc.inverse_transform(pd.concat([df_cur, pred], axis=1))[target_column]

        # model performance over whole dataset
        tmp = df.loc[(df['model'] == model) & (df['dataset'] == dataset), 'accuracy']
        assert tmp.shape[0] == 1
        perf = tmp.iloc[0]
        
        # assemble table
        data.append({
            'model': model,
            'prediction': pred,
            'performance': perf
        })
    
    return data

# run app
app.run_server(mode='jupyterlab')